# Import Data

In [35]:
import pandas as pd

In [36]:
df = pd.read_csv('C:/Users/User/Documents/le wagon project/coronanet_release_allvars.csv', encoding='mac_roman')

# keep the important columns

In [37]:
 def tristans_columns(self):
        df = self.copy()
        df['date_announced'] = pd.to_datetime(df['date_announced'])
        df['date_start'] = pd.to_datetime(df['date_start'])
        df['date_end'] = pd.to_datetime(df['date_end'])
        df['date_announced'].fillna(df['date_start'], inplace = True)
        df['date_start'].clip(lower=df['date_announced'], inplace=True)
        df_select = df[['policy_id', 'date_start', 'country',
           'type', 'type_sub_cat','description', 'ISO_A3','compliance']]
        return df_select

In [38]:
df= tristans_columns(df)

In [39]:
df.isnull().sum(axis = 0)

policy_id           0
date_start          0
country             0
type                0
type_sub_cat    13819
description         0
ISO_A3            243
compliance        244
dtype: int64

In [40]:
df = df.dropna()

In [41]:
df.isnull().sum(axis = 0)

policy_id       0
date_start      0
country         0
type            0
type_sub_cat    0
description     0
ISO_A3          0
compliance      0
dtype: int64

In [42]:
df

,policy_id,date_start,country,type,type_sub_cat,description,ISO_A3,compliance
4,6721435,2020-03-09,Afghanistan,Closure and Regulation of Schools,Preschool or childcare facilities (generally f...,"""All schools in Herat ñ including temporarily ...",AFG,Mandatory (Unspecified/Implied)
5,6721435,2020-03-09,Afghanistan,Closure and Regulation of Schools,Primary Schools (generally for children ages 1...,"""All schools in Herat ñ including temporarily ...",AFG,Mandatory (Unspecified/Implied)
6,6721435,2020-03-09,Afghanistan,Closure and Regulation of Schools,Secondary Schools (generally for children ages...,"""All schools in Herat ñ including temporarily ...",AFG,Mandatory (Unspecified/Implied)
7,6018486,2020-03-11,Afghanistan,Closure and Regulation of Schools,Primary Schools (generally for children ages 1...,"In parts of northern Samangan province, school...",AFG,Mandatory (Unspecified/Implied)
8,6018486,2020-03-11,Afghanistan,Closure and Regulation of Schools,Secondary Schools (generally for children ages...,"In parts of northern Samangan province, school...",AFG,Mandatory (Unspecified/Implied)
...,...,...,...,...,...,...,...,...
45635,1599366,2020-09-17,Zimbabwe,Restrictions of Mass Gatherings,Postponement of an annually recurring event,"In Zimbabwe, restriction of non-essential busi...",ZWE,Mandatory (Unspecified/Implied)
45636,1599366,2020-09-17,Zimbabwe,Restrictions of Mass Gatherings,Cancellation of a recreational or commercial e...,"In Zimbabwe, restriction of non-essential busi...",ZWE,Mandatory (Unspecified/Implied)
45637,1599366,2020-09-17,Zimbabwe,Restrictions of Mass Gatherings,Postponement of a recreational or commercial e...,"In Zimbabwe, restriction of non-essential busi...",ZWE,Mandatory (Unspecified/Implied)
45638,1599366,2020-09-17,Zimbabwe,Restrictions of Mass Gatherings,Other mass gatherings not specified above rest...,"In Zimbabwe, restriction of non-essential busi...",ZWE,Mandatory (Unspecified/Implied)


# Tokenization and lemmatization

In [43]:
df["type_and_description"] = df["type"]+' '+df["description"]

In [44]:
df["type_and_description"]

4        Closure and Regulation of Schools "All schools...
5        Closure and Regulation of Schools "All schools...
6        Closure and Regulation of Schools "All schools...
7        Closure and Regulation of Schools In parts of ...
8        Closure and Regulation of Schools In parts of ...
                               ...                        
45635    Restrictions of Mass Gatherings In Zimbabwe, r...
45636    Restrictions of Mass Gatherings In Zimbabwe, r...
45637    Restrictions of Mass Gatherings In Zimbabwe, r...
45638    Restrictions of Mass Gatherings In Zimbabwe, r...
45639    Restrictions of Mass Gatherings The Zimbabwe m...
Name: type_and_description, Length: 31487, dtype: object

In [45]:
import numpy as np
import pandas as pd
import datetime
import pyreadr
import pandas as pd
from IPython.display import display
from sklearn.preprocessing import OneHotEncoder
from nltk.corpus import stopwords 
import string
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import word_tokenize 
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer

In [46]:
def clean(x):
    x = x.lower()
    x = ''.join([letter for letter in x if not letter.isdigit()])
    for p in string.punctuation:
        x = x.replace(p, " ")
    stops = set(stopwords.words("english"))
    tokens = word_tokenize(x)
    tokens = ' '.join([token for token in tokens if token not in stops])
    return tokens

In [47]:
df['clean_description'] = df.type_and_description.apply(clean)

In [48]:
df['clean_description']

4        closure regulation schools schools herat ñ inc...
5        closure regulation schools schools herat ñ inc...
6        closure regulation schools schools herat ñ inc...
7        closure regulation schools parts northern sama...
8        closure regulation schools parts northern sama...
                               ...                        
45635    restrictions mass gatherings zimbabwe restrict...
45636    restrictions mass gatherings zimbabwe restrict...
45637    restrictions mass gatherings zimbabwe restrict...
45638    restrictions mass gatherings zimbabwe restrict...
45639    restrictions mass gatherings zimbabwe minister...
Name: clean_description, Length: 31487, dtype: object

In [49]:
def lemma(text):
    lemmatizer = WordNetLemmatizer()
    return [lemmatizer.lemmatize(w) for w in text]

In [50]:
df['clean_description'] = lemma(df['clean_description'])

In [51]:
from nltk.stem.porter import PorterStemmer

stemmer = PorterStemmer()

df['clean_description'] = [stemmer.stem(word) for word in df['clean_description']]


# Now lets encode my subcat

In [52]:
df[['type_sub_cat']]

,type_sub_cat
4,Preschool or childcare facilities (generally f...
5,Primary Schools (generally for children ages 1...
6,Secondary Schools (generally for children ages...
7,Primary Schools (generally for children ages 1...
8,Secondary Schools (generally for children ages...
...,...
45635,Postponement of an annually recurring event
45636,Cancellation of a recreational or commercial e...
45637,Postponement of a recreational or commercial e...
45638,Other mass gatherings not specified above rest...


In [53]:
temp = df['type_sub_cat'].value_counts()

In [54]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit_transform(list(pd.DataFrame(temp).reset_index()['index']))

array([ 86,  48,  65,  87,  79,  64,  28,  11,  14,  85,  88,  59,  41,
        49,   4,  51,   1,  36,  50,  21,  29,  44,  60,  55,  98,  99,
         9,  53,   3,  15,  34,  26, 110,  81,  52,  93,  56,  68,  82,
        96,  83,  45,  80,  91,  89, 114, 116, 104, 117, 106,  31,  22,
        92,  47,  54,  75,  97,  74,  10,  18,  23,  30,  42,  17,  67,
        61,  71,  27,   6,  62,  95,  46,  70,  57,  25,  69, 118,   2,
        77, 103,  63, 100,  58,  19,  76,  12,   5,  24,  20,  37,   0,
       105, 102,  84,  16,  13,  94,  73,  66,  32,  90,  35, 107,  72,
        43,   7,  33, 101,   8,  38, 108, 115, 111, 109,  40,  78,  39,
       113, 119, 112], dtype=int64)

In [55]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(df['type_sub_cat'])
le_name_mapping = dict(zip(le.classes_, le.transform(le.classes_)))


In [56]:
df['encoded_sub-cat']= df['type_sub_cat'].map(le_name_mapping)

In [86]:
df['encoded_sub-cat']

4        64
5        65
6        86
7        65
8        86
         ..
45635    63
45636     9
45637    62
45638    55
45639     3
Name: encoded_sub-cat, Length: 31487, dtype: int64

# Now we can run a Naive Bayes model and see its score

In [58]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['clean_description']) 
y = df['encoded_sub-cat']

In [59]:
from sklearn.naive_bayes import MultinomialNB

nb_model = MultinomialNB()

nb_model.fit(X,y)

nb_model.score(X,y)

0.3131133483659923

In [60]:
A = nb_model.predict(vectorizer.fit_transform(df["clean_description"])[31287:])

B =df['encoded_sub-cat'][31287:]

C = A==B

C.value_counts()

False    145
True      55
Name: encoded_sub-cat, dtype: int64

In [61]:
55/200

0.275

How do I increase my score ? Need to do a nice grid search, maybe find a way to regroup some of the sub-cats, maybe only take the top 3 results, then REDO a naive Bayse pred from those => OVER THE WEEKEND

## Gonna try with a pipeline and Gridsearch now

In [35]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

# Create Pipeline
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('nb', MultinomialNB()),
])

# Set parameters to search
parameters = {
    'tfidf__ngram_range': ((1,1),(2,2)),
    'tfidf__max_features': (1000,2000,3000),
    'tfidf__max_df': (1,0.9,0.8,0.7),
    'nb__alpha': (0,0.1)}

# Perform grid search
grid_search = GridSearchCV(pipeline, parameters, n_jobs=1, 
                           verbose=1, scoring = "accuracy", 
                           refit=True, cv=5)




In [36]:
#grid_search.fit(X,y)

In [37]:
grid_search.best_params_

AttributeError: 'GridSearchCV' object has no attribute 'best_params_'

In [ ]:
grid_search.best_score_

After multiple gridsearches, I realised this wasnt working so I stuck to this one for now (I'm testing it on the last 200 subcats here):

In [62]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['clean_description'])[:31287]
y = df['encoded_sub-cat'][:31287]

In [63]:
from sklearn.naive_bayes import MultinomialNB

nb_model = MultinomialNB()

nb_model.fit(X,y)

nb_model.score(X,y)

0.31345287179978903

In [65]:
A = nb_model.predict(vectorizer.fit_transform(df['clean_description'])[31287:])

In [66]:
B =df['encoded_sub-cat'][31287:]

In [67]:
C = A==B

In [68]:
C.value_counts()

False    153
True      47
Name: encoded_sub-cat, dtype: int64

## 47 true / 200 (keep that in mind for later)

# Now im gonna try a SGDC model to see if it can give me a better pred

In [72]:
from sklearn.linear_model import SGDClassifier

In [73]:
SGDC = SGDClassifier()
#No params for now
# loss =‘hinge’, ‘log’, ‘modified_huber’, ‘squared_hinge’, ‘perceptron’, or a regression loss: ‘squared_loss’, ‘huber’, ‘epsilon_insensitive’, or ‘squared_epsilon_insensitive’

In [74]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df["clean_description"])[:31287]
y = df['encoded_sub-cat'][:31287]

In [75]:
SGDC.fit(X,y)

c:\users\user\.venvs\lewagon\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


SGDClassifier(alpha=0.0001, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='hinge', max_iter=None,
       n_iter=None, n_iter_no_change=5, n_jobs=None, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False)

In [76]:
SGDC.score(X,y)

0.5446990762936683

55%...Ill tryn it on the last 200 rows

In [78]:
A = SGDC.predict(vectorizer.fit_transform(df["clean_description"])[31287:])

In [79]:
B =df['encoded_sub-cat'][31287:]

In [80]:
C = A==B

In [81]:
C.value_counts()

False    127
True      73
Name: encoded_sub-cat, dtype: int64

In [82]:
73/200

0.365

# WELL..

## Now i need to play with the params to get the best pred, this might take some time...

In [156]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV

# Create Pipeline
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('SGDC', SGDClassifier()),
])

# Set parameters to search
parameters = {
    'tfidf__max_features': (1000,5000,10000),
    'tfidf__max_df': (0.6, 0.4, 0.2),
    'SGDC__loss': ('modified_huber', 'squared_loss', 'huber', 'epsilon_insensitive'),
    'SGDC__alpha': (0.001, 0.005, 0.007)}

# Perform grid search
grid_search = GridSearchCV(pipeline, parameters, n_jobs=1,verbose = 1,
                         scoring = "accuracy", 
                           refit=True, cv=5)

In [157]:
X =df["clean_description"]
y = df['encoded_sub-cat']

In [158]:
grid_search.fit(X,y)

Fitting 5 folds for each of 108 candidates, totalling 540 fits


c:\users\user\.venvs\lewagon\lib\site-packages\sklearn\model_selection\_split.py:651: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
c:\users\user\.venvs\lewagon\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)
[Parallel(n_jobs=1)]: Done 540 out of 540 | elapsed: 82.1min finished
c:\users\user\.venvs\lewagon\lib\site-packages\sklearn\model_selection\_search.py:842: DeprecationWarning: The default of the `iid` parameter will change from True to False in 

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,...m_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False))]),
       fit_params=None, iid='warn', n_jobs=1,
       param_grid={'tfidf__max_features': (1000, 5000, 10000), 'tfidf__max_df': (0.6, 0.4, 0.2), 'SGDC__loss': ('modified_huber', 'squared_loss', 'huber', 'epsilon_insensitive'), 'SGDC__alpha': (0.001, 0.005, 0.007)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=1)

In [159]:
grid_search.best_params_

{'SGDC__alpha': 0.001,
 'SGDC__loss': 'modified_huber',
 'tfidf__max_df': 0.6,
 'tfidf__max_features': 5000}

In [161]:
grid_search.best_score_

0.3540191190014927

In [162]:
vectorizer = TfidfVectorizer(max_df =  0.6, max_features = 5000)
X = vectorizer.fit_transform(df['clean_description'])
y = df['encoded_sub-cat']

In [163]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=1)

In [164]:
SGDC = SGDClassifier(alpha =  0.001, loss = 'modified_huber')
SGDC.fit(X_train,y_train)

SGDClassifier(alpha=0.001, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='modified_huber',
       max_iter=None, n_iter=None, n_iter_no_change=5, n_jobs=None,
       penalty='l2', power_t=0.5, random_state=None, shuffle=True,
       tol=None, validation_fraction=0.1, verbose=0, warm_start=False)

In [165]:
SGDC.score(X_train,y_train)

0.49799515661598315

In [166]:
A = SGDC.predict(X_test)
B = y_test

C= A==B

C.value_counts()

False    4047
True     2251
Name: encoded_sub-cat, dtype: int64

In [167]:
2251/6298 * 100

35.74150523975865

## Guess I cant get higher than this..